In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report

# Define model directories (paths)
model_dirs = {
    'T1': '/mnt/nrdstor/amilab/towfiqami/Final/codes/RESNET/All-train-test/T1_best_model.h5',
    'T2': '/mnt/nrdstor/amilab/towfiqami/Final/codes/RESNET/All-train-test/T2_best_model.h5',
    'T3': '/mnt/nrdstor/amilab/towfiqami/Final/codes/RESNET/All-train-test/T3_best_model.h5',
    'HL1': '/mnt/nrdstor/amilab/towfiqami/Final/codes/RESNET/All-train-test/HL1_best_model.h5',
    'HL2': '/mnt/nrdstor/amilab/towfiqami/Final/codes/RESNET/All-train-test/HL2_best_model.h5',
    'data': '/mnt/nrdstor/amilab/towfiqami/Final/codes/RESNET/All-train-test/data_best_model.h5'
}

# Define test directories (paths)
test_dirs = {
    'T1': '/mnt/nrdstor/amilab/towfiqami/Final/DATA/data12-15-2024/BLOCK/test1/crate_type/TSL',
    'T2': '/mnt/nrdstor/amilab/towfiqami/Final/DATA/data12-15-2024/BLOCK/test1/crate_type/ECSL',
    'T3': '/mnt/nrdstor/amilab/towfiqami/Final/DATA/data12-15-2024/BLOCK/test1/crate_type/ESCSL',
    'HL1': '/mnt/nrdstor/amilab/towfiqami/Final/DATA/data12-15-2024/BLOCK/test1/heat_lamp/1',
    'HL2': '/mnt/nrdstor/amilab/towfiqami/Final/DATA/data12-15-2024/BLOCK/test1/heat_lamp/1',
    'data': '/mnt/nrdstor/amilab/towfiqami/Final/DATA/data12-15-2024/DATA/test1'
}

# Define test data generator
datagen_test = ImageDataGenerator(rescale=1./255)

# Directory for outputs
output_dir = "/mnt/nrdstor/amilab/towfiqami/Final/codes/RESNET/confusion"
os.makedirs(output_dir, exist_ok=True)

# Function to evaluate a model on a test dataset
def evaluate_model_on_test(model_path, test_path, model_name, test_name):
    print(f"Evaluating Model '{model_name}' on Test Dataset '{test_name}'...")
    
    # Load model
    model = load_model(model_path)
    
    # Load test data
    test_generator = datagen_test.flow_from_directory(
        test_path,
        target_size=(220, 220),
        batch_size=32,  # Fixed batch size
        class_mode='categorical',
        shuffle=False
    )
    
    # Class labels
    class_labels = list(test_generator.class_indices.keys())
    
    # Predict
    STEP_SIZE_TEST = np.ceil(test_generator.samples / test_generator.batch_size)
    y_pred = model.predict(test_generator, steps=STEP_SIZE_TEST, verbose=1)
    y_pred = np.argmax(y_pred, axis=1)
    
    # Confusion matrix
    conf_mat = confusion_matrix(test_generator.classes, y_pred)
    conf_mat_norm = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]
    
    # Save confusion matrix as a plot
    plt.figure(figsize=(8, 6))
    sn.set(font_scale=1)
    for i in range(conf_mat.shape[0]):
        for j in range(conf_mat.shape[1]):
            text = f"{conf_mat_norm[i, j]:.2f}\n{conf_mat[i, j]}"
            color = 'black' if conf_mat_norm[i, j] < 0.5 else 'white'
            plt.text(j + 0.5, i + 0.5, text, ha='center', va='center', color=color)
    sn.heatmap(conf_mat_norm, annot=False, fmt='.2f', cmap='Blues', cbar=True,
               xticklabels=class_labels, yticklabels=class_labels)
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    confusion_matrix_path = os.path.join(output_dir, f"{model_name}_on_{test_name}_confusion_matrix.png")
    plt.savefig(confusion_matrix_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    # Classification report
    report = classification_report(test_generator.classes, y_pred, target_names=class_labels, output_dict=True)
    classification_report_path = os.path.join(output_dir, f"classification_report_{model_name}_on_{test_name}.csv")
    
    # Convert report to a DataFrame and save as CSV
    report_df = pd.DataFrame(report).transpose()
    report_df['Model'] = model_name
    report_df['Test'] = test_name
    report_df.to_csv(classification_report_path, index=True)

# Evaluate all model-test combinations
combinations = [
    ('data', 'data'),
    ('T1', 'T1'),
    ('T1', 'T2'),
    ('T1', 'T3'),
    ('T2', 'T1'),
    ('T2', 'T2'),
    ('T2', 'T3'),
    ('T3', 'T1'),
    ('T3', 'T2'),
    ('T3', 'T3'),
    ('HL1', 'HL1'),
    ('HL1', 'HL2'),
    ('HL2', 'HL1'),   
    ('HL2', 'HL2')
]

for model_name, test_name in combinations:
    evaluate_model_on_test(model_dirs[model_name], test_dirs[test_name], model_name, test_name)

print(f"Evaluation complete. Confusion matrices and classification reports saved in: {output_dir}")


Evaluating Model 'data' on Test Dataset 'data'...


2024-12-16 15:37:22.039205: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-16 15:37:23.754801: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30958 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:5e:00.0, compute capability: 7.0


Found 17802 images belonging to 6 classes.


2024-12-16 15:37:33.750434: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401
2024-12-16 15:37:35.466830: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-12-16 15:37:35.468574: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-12-16 15:37:35.468589: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2024-12-16 15:37:35.468934: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-12-16 15:37:35.468980: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


557/557 [==============================] - 399s 709ms/step
Evaluating Model 'T1' on Test Dataset 'T1'...
Found 5825 images belonging to 6 classes.
183/183 [==============================] - 174s 955ms/step
Evaluating Model 'T1' on Test Dataset 'T2'...
Found 5665 images belonging to 6 classes.
178/178 [==============================] - 155s 870ms/step
Evaluating Model 'T1' on Test Dataset 'T3'...
Found 6312 images belonging to 6 classes.
198/198 [==============================] - 156s 791ms/step
Evaluating Model 'T2' on Test Dataset 'T1'...
Found 5825 images belonging to 6 classes.
183/183 [==============================] - 18s 92ms/step
Evaluating Model 'T2' on Test Dataset 'T2'...
Found 5665 images belonging to 6 classes.
178/178 [==============================] - 18s 98ms/step
Evaluating Model 'T2' on Test Dataset 'T3'...
Found 6312 images belonging to 6 classes.
198/198 [==============================] - 20s 99ms/step
Evaluating Model 'T3' on Test Dataset 'T1'...
Found 5825 images b

In [3]:
import os
import pandas as pd

# Directory containing the individual CSV files
input_dir = "/mnt/nrdstor/amilab/towfiqami/Final/codes/RESNET/confusion"
output_file = os.path.join(input_dir, "combined_classification_reports.csv")

# Initialize an empty list to store DataFrames
combined_data = []

# Iterate over all files in the directory
for file in os.listdir(input_dir):
    if file.endswith(".csv"):  # Check if the file is a CSV
        file_path = os.path.join(input_dir, file)
        print(f"Processing file: {file_path}")
        # Read CSV into a DataFrame
        df = pd.read_csv(file_path)
        combined_data.append(df)

# Combine all DataFrames into a single DataFrame
combined_df = pd.concat(combined_data, ignore_index=True)

# Save the combined DataFrame to a single CSV file
combined_df.to_csv(output_file, index=False)

print(f"Combined CSV saved at: {output_file}")


Processing file: /mnt/nrdstor/amilab/towfiqami/Final/codes/RESNET/confusion/classification_report_HL2_on_HL1.csv
Processing file: /mnt/nrdstor/amilab/towfiqami/Final/codes/RESNET/confusion/classification_report_T3_on_T1.csv
Processing file: /mnt/nrdstor/amilab/towfiqami/Final/codes/RESNET/confusion/classification_report_data_on_data.csv
Processing file: /mnt/nrdstor/amilab/towfiqami/Final/codes/RESNET/confusion/classification_report_T1_on_T3.csv
Processing file: /mnt/nrdstor/amilab/towfiqami/Final/codes/RESNET/confusion/classification_report_T2_on_T2.csv
Processing file: /mnt/nrdstor/amilab/towfiqami/Final/codes/RESNET/confusion/classification_report_T2_on_T3.csv
Processing file: /mnt/nrdstor/amilab/towfiqami/Final/codes/RESNET/confusion/classification_report_T3_on_T3.csv
Processing file: /mnt/nrdstor/amilab/towfiqami/Final/codes/RESNET/confusion/classification_report_T2_on_T1.csv
Processing file: /mnt/nrdstor/amilab/towfiqami/Final/codes/RESNET/confusion/classification_report_T3_on_T2